<a href="https://colab.research.google.com/github/jhogarciacu/TAM_2025_2/blob/main/tanjiro_4-7_(Reducci%C3%B3n_de_caracteristicas_y_clusterin).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://drive.google.com/file/d/1KENM6BW5UjRoSjnWmuBP_MS_NrC0cefP/view?usp=sharing

FILEID = "1KENM6BW5UjRoSjnWmuBP_MS_NrC0cefP"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O tanjiro.jpeg && rm -rf /tmp/cookies.txt

In [ ]:
!pip install umap-learn

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN,SpectralClustering

In [ ]:
image = imread(os.path.join("tanjiro.jpeg"))
image = image[50:175,50:175,:]
plt.imshow(image)
image.shape

In [ ]:
X = image.reshape(-1, 3)
r,c,ch = image.shape
print(r,c,ch)

In [ ]:
vr = np.linspace(0,r,r,endpoint=False)
vc = np.linspace(0,c,c,endpoint=False)
Xc = np.kron(np.ones((r,1)),vc.reshape(-1,1))
Xr = np.kron(vr.reshape(-1,1),np.ones((1,c)))
Xc = Xc.reshape(-1)
Xr = Xr.reshape(-1)
X = np.c_[X,Xc,Xr]
X.shape

In [ ]:
plt.pcolormesh(X)
plt.colorbar()
plt.show()

In [ ]:
# Escalamos valores para que esten en las ismas dimensiones
scaler = MinMaxScaler()
X_ = scaler.fit_transform(X)

# Reduccion de dimensiones

In [ ]:
# PCA
X_pca = PCA(n_components=2).fit_transform(X_)
plt.scatter(X_pca[:,0], X_pca[:,1], c=X[:,2])
plt.show()

In [ ]:
# UMAP
X_umap = UMAP(n_components=2,n_neighbors = 30,#round(np.sqrt(X_.shape[0])),
          min_dist =0.9).fit_transform(X_)


# **Tipos de clustering aplicados**

### K-means

In [ ]:
kmeans = KMeans(n_clusters=4,n_init='auto').fit(X_)
kmeans.labels_

### K-means++

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++', n_init='auto').fit(X_)
labels_kmeans = kmeans.labels_

### Accelerated K-Means (Elkan)

In [ ]:
kmeans_elkan = KMeans(n_clusters=4, algorithm='elkan', n_init='auto').fit(X_)
labels_elkan = kmeans_elkan.labels_

### MiniBatch K-Means

In [ ]:
minibatch = MiniBatchKMeans(n_clusters=4, batch_size=100).fit(X_)
labels_minibatch = minibatch.labels_

### DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5).fit(X_)
labels_dbscan = dbscan.labels_

### Spectral Clustering

In [ ]:
spectral = SpectralClustering(n_clusters=4, affinity='nearest_neighbors').fit(X_)
labels_spectral = spectral.labels_

In [ ]:
def plot_clusters(X_, labels, title):
    plt.figure(figsize=(6, 5))
    plt.scatter(X_[:, 0], X_[:, 1], c=labels, cmap='tab10', s=20)
    plt.title(title)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.show()

plot_clusters(X_, kmeans.labels_, "K-Means")
plot_clusters(X_, labels_kmeans, "K-Means++")
plot_clusters(X_, labels_elkan, "Accelerated K-Means (Elkan)")
plot_clusters(X_, labels_minibatch, "MiniBatch K-Means")
plot_clusters(X_, labels_dbscan, "DBSCAN")
plot_clusters(X_, labels_spectral, "Spectral Clustering")


# Visualizacion con imagen procesada


In [ ]:
def plot_clusters_image(labels, title):
  labels_ = labels
  Ilabels = labels_.reshape(r,c)
  plt.imshow(Ilabels)
  plt.colorbar()
  plt.title(title)
  plt.xlabel('UMAP 1')
  plt.ylabel('UMAP 2')
  plt.show()


plot_clusters_image(kmeans.labels_, "K-Means")
plot_clusters_image(labels_kmeans, "K-Means++")
plot_clusters_image(labels_elkan, "Accelerated K-Means (Elkan)")
plot_clusters_image(labels_minibatch, "MiniBatch K-Means")
plot_clusters_image(labels_dbscan, "DBSCAN")
plot_clusters_image(labels_spectral, "Spectral Clustering")

In [ ]:
D_ = kmeans.transform(X_)
D_.shape
ind_ = np.argmin(D_,axis=0)

In [ ]:
D_.shape
ind_

In [ ]:
centroids_ = X[ind_,:3]
centroids_

In [ ]:
segmented_img = centroids_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)
plt.imshow(segmented_img/255.)

plt.show()

In [ ]:
centroids_[kmeans.labels_]

In [ ]:
kmeans.labels_

In [ ]:
def evaluar_clustering(X_, modelos, k_values=None, random_state=42):

    resultados_globales = []
    mejor_silhouette = -1
    mejor_modelo = None
    mejor_labels = None
    mejor_nombre = None
    mejor_k = None

    for nombre, constructor in modelos.items():
        print(f"\n🔹 Evaluando modelo: {nombre}")
        resultados = []

        # Si el modelo usa k (tiene parámetro n_clusters)
        if k_values:
            for k in k_values:
                modelo = constructor(k)
                labels = modelo.fit_predict(X_)

                if hasattr(modelo, 'inertia_'):
                    inertia = modelo.inertia_
                else:
                    inertia = np.nan

                if len(set(labels)) > 1:
                    silhouette = silhouette_score(X_, labels)
                else:
                    silhouette = np.nan

                resultados.append({'Modelo': nombre, 'k': k, 'Inercia': inertia, 'Silhouette': silhouette})

                # Guardar el mejor modelo global
                if silhouette > mejor_silhouette:
                    mejor_silhouette = silhouette
                    mejor_modelo = modelo
                    mejor_labels = labels
                    mejor_nombre = nombre
                    mejor_k = k

        else:  # Para modelos sin parámetro de k (como DBSCAN)
            modelo = constructor(None)
            labels = modelo.fit_predict(X_)
            inertia = np.nan
            silhouette = silhouette_score(X_, labels) if len(set(labels)) > 1 else np.nan
            resultados.append({'Modelo': nombre, 'k': np.nan, 'Inercia': inertia, 'Silhouette': silhouette})

            if silhouette > mejor_silhouette:
                mejor_silhouette = silhouette
                mejor_modelo = modelo
                mejor_labels = labels
                mejor_nombre = nombre
                mejor_k = None

        df_modelo = pd.DataFrame(resultados)
        resultados_globales.append(df_modelo)
        print(df_modelo)

    # Combinar todos los resultados
    df_resultados = pd.concat(resultados_globales, ignore_index=True)

    # Graficar comparación de Silhouette entre modelos
    plt.figure(figsize=(8, 4))
    for nombre in df_resultados['Modelo'].unique():
        subset = df_resultados[df_resultados['Modelo'] == nombre]
        plt.plot(subset['k'], subset['Silhouette'], 'o-', label=nombre)
    plt.xlabel('Número de Clusters (k)')
    plt.ylabel('Coeficiente de Silueta')
    plt.title('Comparación de Silhouette entre Modelos')
    plt.legend()
    plt.show()

    # Graficar el mejor resultado encontrado
    plt.figure(figsize=(6, 5))
    plt.scatter(X_[:, 0], X_[:, 1], c=mejor_labels, cmap='tab10', s=20)
    titulo = f"🏆 Mejor modelo: {mejor_nombre}"
    if mejor_k:
        titulo += f" (k={mejor_k})"
    titulo += f"\nSilhouette={mejor_silhouette:.3f}"
    plt.title(titulo)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.show()

    return mejor_modelo, mejor_nombre, mejor_k, df_resultados


In [ ]:
modelos = {
    "K-Means++": lambda k: KMeans(n_clusters=k, init='k-means++', random_state=42),
    "MiniBatch K-Means": lambda k: MiniBatchKMeans(n_clusters=k, random_state=42),
    "Spectral Clustering": lambda k: SpectralClustering(n_clusters=k, random_state=42),
    "DBSCAN": lambda k: DBSCAN(eps=0.3, min_samples=5)
}

# Buscamos mejor clustering sin reducción
mejor_modelo, mejor_nombre, mejor_k, resultados = evaluar_clustering(X_, modelos, k_values=[3, 5, 7, 9])


In [ ]:
# Buscamos mejor clustering con PCA
mejor_modelo, mejor_nombre, mejor_k, resultados = evaluar_clustering(X_pca, modelos, k_values=[3, 5, 7, 9])


In [ ]:
# Buscamos mejor clustering con UMAP
mejor_modelo, mejor_nombre, mejor_k, resultados = evaluar_clustering(X_umap, modelos, k_values=[3, 5, 7, 9])
